In [157]:
import pandas as pd
import plotly.express as px
import numpy as np

df = pd.read_csv('data.CSV', header=0, index_col=1)
max_distance = 180000
df = df[df['distance'] <= max_distance]
df.sort_values(by=['search', 'distance'], ascending=[True,True], inplace=True)
df['regression_price'] = np.nan
for car_model in df['search'].unique():
    df_car_model = df.loc[(df['search'] == car_model) & (df['price'] > 0) & (df['price'] < 10000000) & (df['distance'] > 0) & (df['distance'] < max_distance)]
    y = df_car_model['price']
    x = df_car_model['distance']
    try:
        result = np.polyfit(x, y, 3)
        # regression_distance = np.linspace(0, max_distance, 1)
        regression_price = np.poly1d(result)
        df.loc[(df['search'] == car_model), 'regression_price'] = df.loc[(df['search'] == car_model), 'distance'].apply(lambda x: regression_price(x))
    except Exception as e:
        print(repr(e))
        continue
fig = px.line(df, x="distance", y="regression_price", color="search", title="finn.no car models")
fig.update_traces(textposition="bottom right")
for car_model in df['search'].unique():
    df_car_model = df.loc[(df['search'] == car_model) & (df['price'] > 0) & (df['price'] < 10000000) & (df['distance'] > 0) & (df['distance'] < max_distance)]
    fig.add_scatter(x=df_car_model['distance'], y=df_car_model['price'], mode='markers', name=car_model)

fig.show(renderer="browser")